In [1]:
COLAB_FLAG = False   # whether running on colab or locally on computer

In [2]:
if COLAB_FLAG:
    !pip install pydicom==2.1.2
    !pip install monai seaborn sentence_transformers
    !git clone 'https://github.com/mille055/AIPI540_individual_project.git'

    


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchvision
import pydicom
#import monai
import pickle
import glob
import sklearn
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression

import seaborn as sns
from datetime import datetime
from sentence_transformers import SentenceTransformer

from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time
import os
import copy
from pprint import pprint
from fastai.basics import delegates
from fastcore.parallel import parallel
from fastcore.utils import gt
from fastcore.foundation import L

# import monai
# from monai.data import DataLoader, ImageDataset
# from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, EnsureType
from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue
import sys
import importlib
import warnings
warnings.filterwarnings("ignore")

In [4]:
#local imports
if COLAB_FLAG:
    sys.path.append('/content/AIPI540_individual_project/scripts/')
    train_datafile = '/content/AIPI540_individual_project/data/trainfiles.csv'
    val_datafile = '/content/AIPI540_individual_project/data/valfiles.csv'
    test_datafile = '/content/AIPI540_individual_project/data/testfiles.csv'

else: # running locally
    sys.path.append('/Users/cmm/Documents/GitHub/AIPI540_individual_project/scripts/')
    train_datafile = '../data/trainfiles.csv'
    val_datafile = '../data/valfiles.csv'
    test_datafile = '../data/testfiles.csv'

### local imports ###
from config import file_dict, feats, feats_to_keep, column_lists, RF_parameters, classes, model_paths
from config import abd_label_dict, val_list, train_val_split_percent, random_seed, data_transforms
from config import sentence_encoder, series_description_column
from utils import *
#from train_pixel_model import train_pix_model, test_pix_model, ImgDataset, image_to_tensor, get_pixel_preds_and_probs
#from train_meta_model import train_fit_parameter_trial, train_meta_model, calc_feature_importances, get_meta_probs, meta_inference
#from train_text_model import train_text_log_model, load_text_data, get_nlp_inference
from NLP.NLP_inference import get_NLP_inference
from NLP.NLP_training import train_NLP_model
from cnn.cnn_dataset import ImgDataset
from cnn.cnn_inference import image_to_tensor, pixel_inference, test_pix_model, load_pixel_model, visualize_results
from cnn.cnn_model import CustomResNet50
from cnn.cnn_data_loaders import get_data_loaders
from metadata.meta_inference import meta_inference, calc_feature_importances
from metadata.meta_training import train_fit_parameter_trial, train_meta_model, evaluate_meta_model
from fusion_model.fus_model import FusionModel
from fusion_model.fus_inference import get_fusion_inference
from model_container import ModelContainer
from process_tree import Processor, write_labels_into_dicom
# from AIPI540_individual_project.scripts.train_pixel_model import train_model
# from AIPI540_individual_project.scripts.train_text_model import load_text_data, train_text_model, list_incorrect_text_predictions
# from AIPI540_individual_project.scripts.utils import *

In [16]:
fusion_model = FusionModel

class PartialFusionModel(nn.Module):
    def __init__(self, model1, model2, num_classes):
        super(PartialFusionModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        #self.model3 = model3
        self.fusion_layer = nn.Linear(num_classes * 2, num_classes)

    def forward(self, x1, x2):
        
        x = torch.cat((x1, x2), dim=0)
        x = self.fusion_layer(x)

        return x
    
partial_fusion_model = PartialFusionModel



In [18]:
with open('../models/fusion_model041623.pkl', 'rb') as file:
    fusion = pickle.load(file)

print(fusion)

FusionModel(
  (model2): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
       

In [ ]:
### fix my fusion model issue:; have to change from a pickled model to saved weights
def save_fusion_model_weights(pickle_path, weight_path):
    with open(pickle_path, 'rb') as file:
        fusion_model = pickle.load(file)
    torch.save(fusion_model.state_dict(), weight_path)

save_fusion_model_weights(model_paths['fusion'], '../models/fusion_saved_weights042123.pth')
save_fusion_model_weights('../models/meta_and_pixel_fusion_model041623.pkl', '../models/fusion_saved_weights_no_nlp042123.pth')

In [ ]:
### in the fusion model creation code:

Xtrain1 = fusion_train_df.meta_probs.values

Xtrain2 = fusion_train_df.pixel_probs.values
Xtrain3 = fusion_train_df.nlp_probs.values

Xtrain1 = np.stack(Xtrain1, axis=0)
Xtrain2 = np.stack(Xtrain2, axis=0)
Xtrain3 = np.stack(Xtrain3, axis=0)

out_features = 19


y_train_fusion = np.array([classes.index(x) for x in fusion_train_df.true])


In [ ]:
# pickling the dataframes which have undergone inference from the base models

# fusion_train_df.to_pickle('../data/fusion_train.pkl')
# fusion_val_df.to_pickle('../data/fusion_val.pkl')
# fusion_test_df.to_pickle('../data/fusion_test.pkl')


In [ ]:
# class FusionModel(nn.Module):
#     def __init__(self, model1, model2, model3, num_classes):
#         super(FusionModel, self).__init__()
#         self.model1 = model1
#         self.model2 = model2
#         self.model3 = model3
#         self.fusion_layer = nn.Linear(num_classes * 3, num_classes)

#     def forward(self, x1, x2, x3):
        
#         x = torch.cat((x1, x2, x3), dim=0)
#         x = self.fusion_layer(x)

#         return x

In [ ]:
class PartialFusionModel(nn.Module):
    def __init__(self, model1, model2, num_classes):
        super(PartialFusionModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        #self.model3 = model3
        self.fusion_layer = nn.Linear(num_classes * 2, num_classes)

    def forward(self, x1, x2):
        
        x = torch.cat((x1, x2), dim=0)
        x = self.fusion_layer(x)

        return x
    

# ## since I merged this into FusionModel with conditions after saving the weights, i need to try to recover the weights into 
# ## the new mdoel
# # load components


# #pmodel = PartialFusionModel(model_pix, meta_model, num_classes=19)

# from config import model_paths
# model_container = ModelContainer()

# with open(model_paths['fusion_no_nlp'], 'rb') as file:
#     partial_fusion_model = pickle.load(file)

# fusion_model_with_partial_weights = FusionModel(models, num_classes=19, include_nlp=False)
# fusion_model_with_partial_weights.fusion_layer.weight = nn.Parameter(partial_fusion_model.fusion_layer.weight)

# # Save only the model weights (state_dict)
# torch.save(fusion_model_with_partial_weights.state_dict(), '../models/fusion_model_no_nlp042123.pth')



In [ ]:
# fusion_model = FusionModel


# num_classes = 19  # Number of classes
# pixel_model = model
# fusion_model = FusionModel(meta_model, pixel_model, NLP_model, num_classes)
# meta_and_pixel_fusion_model = PartialFusionModel(meta_model, pixel_model, num_classes)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(fusion_model.parameters(), lr=0.001)
# p_optimizer = torch.optim.Adam(meta_and_pixel_fusion_model.parameters(), lr=0.001)

In [ ]:
# num_epochs = 20
# batch_size = 32

# for epoch in range(num_epochs):
#     for i in range(0, len(y_train_fusion), batch_size):
#         actual_batch_size = min(batch_size, len(Xtrain1) - i)
#         X1_batch = torch.tensor(np.array(Xtrain1[i:i+actual_batch_size], dtype=np.float32), dtype=torch.float32)
#         X2_batch = torch.tensor(np.array(Xtrain2[i:i+actual_batch_size], dtype=np.float32), dtype=torch.float32)
#         X3_batch = torch.tensor(np.array(Xtrain3[i:i+actual_batch_size], dtype=np.float32), dtype=torch.float32)
#         y_batch = torch.tensor(y_train_fusion[i:i+actual_batch_size], dtype=torch.long)

#         # print(f"X1_batch shape: {X1_batch.shape}")
#         # print(f"X2_batch shape: {X2_batch.shape}")
#         # print(f"X3_batch shape: {X3_batch.shape}")
#         # print(f"y_batch shape: {y_batch.shape}")


#         optimizer.zero_grad()
#         outputs = fusion_model(X1_batch, X2_batch, X3_batch)
#         loss = criterion(outputs, y_batch)
#         loss.backward()
#         optimizer.step()

#     print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')


In [ ]:
# Xtest1 = fusion_test_df.meta_probs.values
# Xtest2 = fusion_test_df.pixel_probs.values
# Xtest3 = fusion_test_df.nlp_probs.values

# Xtest1 = np.stack(Xtest1, axis=0)
# Xtest2 = np.stack(Xtest2, axis=0)
# Xtest3 = np.stack(Xtest3, axis=0)

# out_features = 19

# actual_classes = [0,2,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,23,25]
# y_test_fusion = np.array([actual_classes.index(x) for x in fusion_test_df.true])

In [ ]:
# import torch.nn.functional as F
# # Evaluate the model with the test data
# with torch.no_grad():
#     X1_test_t = torch.tensor(Xtest1, dtype=torch.float32)
#     X2_test_t = torch.tensor(Xtest2, dtype=torch.float32)
#     X3_test_t = torch.tensor(Xtest3, dtype=torch.float32)
#     y_test_t = torch.tensor(y_test_fusion, dtype=torch.long)

#     outputs = fusion_model(X1_test_t, X2_test_t, X3_test_t)
#     f_probabilities = F.softmax(outputs, dim=1)  # Apply softmax to the outputs

#     _, f_predicted = torch.max(outputs, 1)
#     correct = (f_predicted == y_test_t).sum().item()
#     f_accuracy = correct / len(y_test_t) * 100

# print(f'Test accuracy: {f_accuracy}%')

In [ ]:
# def process_batch(df, data_dir, destination_folder, write_labels=True):

#     df1 = df.copy()
#     #print('In batch, columns are: ', df1.columns)
#     batch = df1.groupby('patientID').apply(lambda x: process_patient(x, data_dir, destination_folder, write_labels))
    
#     # print('writing labels into dicom in location ', dest_name)
#     # for filename in batch.fname:

#     return batch

# def process_patient(patient_df, data_dir, destination_folder, write_labels):
#     processed_exams = patient_df.groupby('exam').apply(lambda x: process_exam(x, data_dir, destination_folder, write_labels))
#     return processed_exams


# def process_exam(exam_df, data_dir, destination_folder, write_labels):
#     # Group exam data by series and apply the process_series function
#     processed_series = exam_df.groupby('series').apply(lambda x: process_series(x, data_dir, destination_folder, write_labels))

#     #result = pd.concat(processed_series)
#     result = processed_series
#     return result
    
# def process_series(series_df, data_dir, destination_folder, write_labels, selection_fraction=0.5):
#     # Sort the dataframe by file_info (or another relevant column)
#     sorted_series = series_df.sort_values(by='fname')

#     # Find the middle image index
#     middle_index = int(len(sorted_series) * selection_fraction)

#     # Get the middle image
#     middle_image = sorted_series.iloc[middle_index]

#     predicted_series_class, predicted_series_confidence = get_fusion_inference(middle_image)

#     sorted_series['predicted_class'] = predicted_series_class
#     sorted_series['prediction_confidence'] = np.round(predicted_series_confidence, 2)

#     #save_path = f'/volumes/cm7/processed/modified/{series_df.patientID}/{series_df.exam}/{series_df["series"]}/'

#      # Define the save path relative to data_dir
#     relative_path = os.path.relpath(series_df.fname.iloc[0], data_dir)
#     save_path = os.path.join(data_dir, destination_folder, os.path.dirname(relative_path))

#     if not os.path.exists(save_path):
#         os.makedirs(save_path)

#     if write_labels:
#         #print('writing new data into', save_path)
#         write_labels_into_dicom(sorted_series, label_num=predicted_series_class,
#                             conf_num=np.round(predicted_series_confidence, 3), path=save_path)

#     return sorted_series

In [ ]:
def process_batch(df, data_dir, destination_folder, write_labels=True):

    df1 = df.copy()
    #print('In batch, columns are: ', df1.columns)
    batch = df1.groupby('patientID').apply(lambda x: process_patient(x, data_dir, destination_folder, write_labels))
    
    # print('writing labels into dicom in location ', dest_name)
    # for filename in batch.fname:

    return batch

def process_patient(patient_df, data_dir, destination_folder, write_labels):
    processed_exams = patient_df.groupby('exam').apply(lambda x: process_exam(x, data_dir, destination_folder, write_labels))
    return processed_exams


def process_exam(exam_df, data_dir, destination_folder, write_labels):
    # Group exam data by series and apply the process_series function
    processed_series = exam_df.groupby('series').apply(lambda x: process_series(x, data_dir, destination_folder))

    #result = pd.concat(processed_series)
    result = processed_series
    return result
    
def process_series(series_df, data_dir, destination_folder, write_labels, selection_fraction=0.5):
    # Sort the dataframe by file_info (or another relevant column)
    sorted_series = series_df.sort_values(by='fname')

    # Find the middle image index
    middle_index = int(len(sorted_series) * selection_fraction)

    # Get the middle image
    middle_image = sorted_series.iloc[middle_index]

    predicted_series_class, predicted_series_confidence = get_fusion_model_prediction(middle_image)

    sorted_series['predicted_class'] = predicted_series_class
    sorted_series['prediction_confidence'] = np.round(predicted_series_confidence, 2)

    #save_path = f'/volumes/cm7/processed/modified/{series_df.patientID}/{series_df.exam}/{series_df["series"]}/'

     # Define the save path relative to data_dir
    relative_path = os.path.relpath(series_df.fname.iloc[0], data_dir)
    save_path = os.path.join(data_dir, destination_folder, os.path.dirname(relative_path))

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    if write_labels:
        #print('writing new data into', save_path)
        write_labels_into_dicom(sorted_series, label_num=predicted_series_class,
                            conf_num=np.round(predicted_series_confidence, 3), path=save_path)

    return sorted_series
    



In [ ]:
def pipeline_new_image_df(data_dir, dest_name='modified', write_labels = True):
    # create the df of image data    
    _, df = get_dicoms(data_dir)
    df1 = df.copy()
    ## manipulate df prior to evaluation
    df1 = expand_filename(df1, ['blank', 'filename', 'series', 'exam', 'patientID'])
    df1.drop(columns='blank', inplace=True)
    df1['file_info']=df1.fname
    df1['img_num'] = df1.file_info.apply(extract_image_number)
    df1['contrast'] = df1.apply(detect_contrast, axis=1)
    df1['plane'] = df1.apply(compute_plane, axis=1)
    df1['series_num'] = df1.series.apply(lambda x: str(x).split('_')[-1])
    #print('columns before preprocess are', df1.columns)

    df1 = preprocess(df1)
   # print('after preprocessin exam is in columns?', ('exam' in df1.columns))
    #print('after preprocessing series is in columns?', ('series' in df1.columns))
    #process the batch of studies
    processed_frame = process_batch(df1, data_dir, dest_name, write_labels)

    return processed_frame


In [ ]:
def select_one_image(series, selection_fraction=0.5):
    sorted_series = series.sort_values()
    #print(sorted_series)
    index = int(len(sorted_series) * selection_fraction)
    print(index, len(sorted_series))
    return sorted_series.iloc[index]

In [ ]:
def mask_one_from_series2(df, selection_fraction=0.5):
    def select_one_image(series):
        sorted_series = series.sort_values()
        index = int(len(sorted_series) * selection_fraction)
        return sorted_series.iloc[index]

    df = df.sort_values(by=['patientID', 'series', 'file_info'])

    grouped_df = df.groupby(['patientID', 'series'])
    selected_rows = grouped_df.agg(select_one_image).reset_index()
    
    return selected_rows

# Example usage
# data = {'patientID': [1, 1, 1, 2, 2, 3, 3, 3, 3],
#         'series': [1, 1, 2, 1, 2, 1, 1, 2, 2],
#         'file_info': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']}

# df = pd.DataFrame(data)
# selected_rows = mask_one_from_series(df, selection_fraction=0.5)
# print(selected_rows)

In [ ]:


# def filter_out_mv(df):
#     # Create a boolean mask to identify rows with MultiValue objects in any column
#     mask = df.applymap(lambda x: isinstance(x, MultiValue)).any(axis=1)
#     #print(df[mask])
#     # Invert the mask to select rows without MultiValue objects
#     rows_without_multivalue = ~mask

#     # Filter out rows with MultiValue objects in any column
#     filtered_df = df[rows_without_multivalue]

#     return mask, filtered_df

#from pydicom.multival import MultiValue
# def compute_plane_new(row):
#     '''
#     Computes the plane of imaging from the direction cosines provided in the `ImageOrientationPatient` field.
#     The format of the values in this field is: `[x1, y1, z1, x2, y2, z2]`,
#     which correspond to the direction cosines for the first row and column of the image pixel data.
#     '''
#     planes = ['sag', 'cor', 'ax']
#     if 'ImageOrientationPatient1' in row.keys():
#         dircos = [v for k, v in row.items() if 'ImageOrientationPatient' in k]
#     else:
#         dircos = row['ImageOrientationPatient']

#         # Handle MultiValue objects by converting them to a list of floats
#         if isinstance(dircos, MultiValue):
#             dircos = [float(x) for x in dircos]

#     # Check if dircos has the expected length
#     if not isinstance(dircos, float) and len(dircos) == 6:
#         dircos = np.array(dircos).reshape(2, 3)
#         pnorm = abs(np.cross(dircos[0], dircos[1]))
#         return planes[np.argmax(pnorm)]
#     else:
#         return 'unknown'

